# 12.4 실습: 파인콘으로 벡터 검색 구현하기

## 12.4.1 파인콘 클라이언트 사용법

- https://www.pinecone.io/
- 인덱스 생성
    1. Create index  
        ![image.png](resources/pinecone-1.png)
    2. Configuration  
        - Vector type: Dense(대부분의 원소가 0이 아닌 값), Sparse(대부분의 원소가 0인 값)  
            효율적인 저장/연산 방식을 적용하기 위해 구분 값을 받음  
        ![image.png](resources/pinecone-2.png)

```python
# 예제 12.7

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = ""
pc = Pinecone(api_key=pinecone_api_key)

pc.create_index("llm-book", spec=ServerlessSpec("aws", "us-east-1"), dimension=768)
index = pc.Index('llm-book')
```

In [ ]:
# 예제 12.8

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
# 임베딩 모델 불러오기
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# 데이터셋 불러오기
klue_dp_train = load_dataset('klue', 'dp', split='train[:100]')

embeddings = sentence_model.encode(klue_dp_train['sentence'])

In [ ]:
# 예제 12.9

# 파이썬 기본 데이터 타입으로 변경
embeddings = embeddings.tolist()
# {"id": 문서 ID(str), "values": 벡터 임베딩(List[float]), "metadata": 메타 데이터(dict) ) 형태로 데이터 준비
insert_data = []
for idx, (embedding, text) in enumerate(zip(embeddings, klue_dp_train['sentence'])):
  insert_data.append({"id": str(idx), "values": embedding, "metadata": {'text': text}})

In [ ]:
# 예제 12.10

upsert_response = index.upsert(vectors = insert_data, namespace='llm-book-sub')

In [ ]:
# 예제 12.11

query_response = index.query(
    namespace='llm-book-sub', # 검색할 네임스페이스
    top_k=10, # 몇 개의 결과를 반환할지
    include_values=True, # 벡터 임베딩 반환 여부
    include_metadata=True, # 메타 데이터 반환 여부
    vector=embeddings[0] # 검색할 벡터 임베딩
)
query_response

In [ ]:
# 예제 12.12

new_text = '변경할 새로운 텍스트'
new_embedding = sentence_model.encode(new_text).tolist()
# 업데이트
update_response = index.update(
    id= '기존_문서_id',
    values=new_embedding,
    set_metadata={'text': new_text},
    namespace='llm-book-sub'
)

# 삭제
delete_response = index.delete(ids=['기존_문서_id'], namespace='llm-book-sub')

## 12.4.2 라마인덱스에서 벡터 데이터베이스 변경하기
- 라마인덱스는 기본 벡터 데이터베이스 외 Pinecoen, Chroma 같은 데이터베이스 사용 가능

In [ ]:
# 예제 12.13

# 파인콘 기본 설정
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)
pc.create_index(
    "quickstart", dimension=1536, metric="euclidean", spec=ServerlessSpec("aws", "us-east-1")
)
pinecone_index = pc.Index("quickstart")

# 라마인덱스에 파인콘 인덱스 연결
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)